# Regressão Linear 

## MQO 

In [1]:
# Inicializando Spark
import findspark
findspark.init('/home/macaubas/spark-3.2.1-bin-hadoop3.2')
import pyspark

from pyspark.sql import SparkSession 

spark = SparkSession.builder.appName("linearReg").getOrCreate()

22/05/11 00:07:39 WARN Utils: Your hostname, macaubas-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
22/05/11 00:07:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/11 00:07:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/11 00:07:41 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/05/11 00:07:41 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/05/11 00:07:41 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


### Imports

In [1]:
# Regressão linear
from pyspark.ml.regression import LinearRegression

# Vetores
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler


ModuleNotFoundError: No module named 'pyspark'

### Carregando os dados

In [54]:
# Load data
data = spark.read.csv('Ecommerce_Customers.csv', inferSchema=True, header=True)

data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



**Alvo da predição: Yearly Amount Spent**

In [55]:
data.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)]

### Criando vetores

In [57]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

**Coluna Label: Yearly Amount Spent**

**Features: Avg Session Length, Time on App, Time on Website, Length of Membership**

In [62]:
# Criando as features (características)
assembler = VectorAssembler(inputCols = ['Avg Session Length',
                                         'Time on App',
                                         'Time on Website',
                                         'Length of Membership'],
                           outputCol='features')


# Transformando os dados
output = assembler.transform(data)

output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [65]:
#Visualizando Features
output.select('Features').head(1)

[Row(Features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [67]:
# Selecionando Dados finais
final_data = output.select('features', 'Yearly Amount Spent')

# Separando entre test e treino
train_data, test_data  = final_data.randomSplit([0.7,0.3])

# Visualizando Treino
train_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[29.5324289670579...|  408.6403510726275|
|[30.5743636841713...| 442.06441375806565|
|[30.7377203726281...|  461.7807421962299|
|[30.8162006488763...|   266.086340948469|
|[30.8364326747734...|  467.5019004269896|
|[30.8794843441274...|  490.2065999848547|
|[31.0472221394875...|  392.4973991890214|
|[31.0662181616375...| 448.93329320767435|
|[31.1239743499119...|  486.9470538397658|
|[31.1280900496166...|  557.2526867470547|
|[31.1695067987115...|  427.3565308022928|
|[31.2681042107507...|  423.4705331738239|
|[31.2834474760581...|  591.7810894256675|
|[31.3091926408918...|  432.7207178399336|
|[31.3123495994443...|  463.5914180279406|
|[31.3662121671876...|  430.5888825564849|
|[31.3895854806643...|  410.0696110599829|
|[31.4252268808548...|  530.7667186547619|
|[31.4459724827577...| 484.87696493512857|
|[31.5147378578019...|  489.8124879964614|
+----------

### Descrevendo treino e teste

In [68]:
# Treino
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                358|
|   mean|  497.6912690904668|
| stddev|  77.48853448017037|
|    min| 256.67058229005585|
|    max|  744.2218671047146|
+-------+-------------------+



In [69]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                142|
|   mean|  503.4052450345657|
| stddev|   83.8865350300295|
|    min|  275.9184206503857|
|    max|  765.5184619388373|
+-------+-------------------+



### Modelo

In [72]:
# Caracterizando o modelo
lr = LinearRegression(labelCol= 'Yearly Amount Spent')

# Fitting model aos dados treinados
lr_Model = lr.fit(train_data)

22/05/11 13:53:15 WARN Instrumentation: [ce8dc637] regParam is zero, which might cause numerical instability and overfitting.


In [73]:
# Coeficientes
lr_Model.coefficients

DenseVector([26.1654, 38.5181, 0.2437, 61.3936])

In [74]:
# Intercepto
lr_Model.intercept

-1055.8160578466643

### Avaliando o modelo

In [75]:
# Checando ajuste
test_results = lr_Model.evaluate(test_data)

In [81]:
# Residuals
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| -10.916118597799937|
|  -4.673331989359042|
|   7.657118079531074|
|  -5.225244167759058|
|-0.01925444706319...|
|   4.834189303509788|
| -6.9869363552396635|
| -3.9815282086972843|
|  -0.987317731026792|
|  18.741245924202076|
|  2.7550282360833194|
|  -3.686020012329436|
|    8.64922443316351|
| -1.8078080953935114|
| -13.358565854058952|
|   8.620630576437293|
|   17.57568631007598|
|   1.402201901403032|
|   5.997185443290505|
| -11.357883217481287|
+--------------------+
only showing top 20 rows



In [82]:
# R2
test_results.r2

0.9869903456063374

In [76]:
# RMSE
test_results.rootMeanSquaredError

9.534337472778349

### Deploying model - apenas features


Baseados nas características, qual a previsão? 

In [83]:
unlabeled_data = test_data.select('features')

unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.3931845423455...|
|[30.4925366965402...|
|[30.9716756438877...|
|[31.0613251567161...|
|[31.2606468698795...|
|[31.3584771924370...|
|[31.4474464941278...|
|[31.5171218025062...|
|[31.5761319713222...|
|[31.6005122003032...|
|[31.7366356860502...|
|[31.8124825597242...|
|[31.8512531286083...|
|[31.8530748017465...|
|[31.9365486184489...|
|[31.9549038566348...|
|[32.0180740106320...|
|[32.0444861274404...|
|[32.0478009788678...|
|[32.0637746203136...|
+--------------------+
only showing top 20 rows



In [86]:
predictions = lr_Model.transform(unlabeled_data)

predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.3931845423455...|330.84498840099354|
|[30.4925366965402...|287.14457770927356|
|[30.9716756438877...|486.98149167736165|
|[31.0613251567161...|492.78070222566066|
|[31.2606468698795...|421.34588570401456|
|[31.3584771924370...| 490.3417611459656|
|[31.4474464941278...| 425.5896784504637|
|[31.5171218025062...|  279.899948859083|
|[31.5761319713222...| 542.2139017203551|
|[31.6005122003032...|460.43160556689486|
|[31.7366356860502...| 494.1784180194486|
|[31.8124825597242...|396.49636499612666|
|[31.8512531286083...| 464.3430222336349|
|[31.8530748017465...| 461.0929315577455|
|[31.9365486184489...|440.55795074938715|
|[31.9549038566348...| 431.3772493634897|
|[32.0180740106320...| 340.2074244352393|
|[32.0444861274404...|446.82762728514695|
|[32.0478009788678...|507.45338574280595|
|[32.0637746203136...| 389.6887901242851|
+--------------------+------------